## Exercise 4
### 1)

In [3]:
import pandas as pd
import numpy as np
from icecream import ic

# Running it 10 times
res = []
for _ in range(100):

    m = 10
    mean_service_time = 8
    mean_time_btw_customers = 1

    cum_cust = 0
    cum_time = 0
    blocked = []
    n_customer = 10_000
    service = []


    def update_service_time(service, time):
        # update the time
        for i in range(len(service)):
            service[i] = service[i] - time

    def remove_serviced(service, t):
        # remove the ones which are below a threshold
        new_service = []
        for i in range(len(service)):
            if service[i] > t:
                new_service.append(service[i])
        return new_service


    for id in range(n_customer):
        time = np.random.exponential(mean_time_btw_customers) 

        # remove the time drawn from the customers "in service" time
        update_service_time(service, time)
        service = remove_serviced(service, 0) # remove the customers with 0 or below time

        cum_cust += 1
        cum_time += time
        if len(service) < m:
            service_time = np.random.exponential(mean_service_time, 1)[0] 
            service.append(service_time)
        else:
            blocked.append(id)
    res.append(len(blocked) / n_customer)

In [53]:
import math
import scipy
# The mean is:

def var_mean(res):
    mean = np.mean(np.array(res))
    var = abs((1/(n_customer -1))  * sum([(x**2)- mean**2 for x in res]))
    first = ((math.sqrt(var))  /  math.sqrt(n_customer))  
    second = scipy.stats.t.ppf(0.025, df = (n_customer-1)) 
    third = scipy.stats.t.ppf(1-0.025, df = (n_customer-1)) 
    ic(mean + first*second)
    ic(mean)
    ic(mean + first*third)
var_mean(res)

ic| mean + first*second: 0.11999999999851683
ic| mean: 0.12000000000000002
ic| mean + first*third: 0.12000000000148321


In [5]:
# Calculating theoretical
import math

l = mean_time_btw_customers
s = mean_service_time
A =  l * s

upp = ((A**m)/math.factorial(m)) 
lowr = [(A**i)/math.factorial(i) for i in range(m + 1)]
lowr = sum(lowr)
upp/lowr

0.12166106425295149

We have nearly the same values between the theroretical and our simulated 

### 2)

In [138]:
import pandas as pd
import numpy as np
from icecream import ic
from functools import partial
# The same algorithm as above 
def algo(func1):
    m = 10
    mean_service_time = 8
    mean_time_btw_customers = 1

    cum_cust = 0
    cum_time = 0
    blocked = []
    n_customer = 10_000
    service = []


    for id in range(n_customer):
        time = func1()

        # remove the time drawn from the customers "in service" time
        update_service_time(service, time)
        service = remove_serviced(service, 0) # remove the customers with 0 or below time

        cum_cust += 1
        cum_time += time
        if len(service) < m:
            service_time = np.random.exponential(mean_service_time, 1)[0] 
            service.append(service_time)
        else:
            blocked.append(id)

    return (len(blocked) / n_customer)

res = []
for _ in range(11):
    out = algo(partial(scipy.stats.erlang.rvs, a=1))
    res.append(out)
var_mean(res)

ic| mean + first*second: 0.12218807207598449
ic| mean: 0.12219090909090907
ic| mean + first*third: 0.12219374610583365


In [143]:


def hyper_exp():
    p = 0.8
    rand = np.random.uniform() 
    if rand  < p:
        return np.random.exponential(1/0.8333, 1)[0] 
    else: 
        return np.random.exponential(1/5.0, 1)[0] 

res = []
for _ in range(11):
    out = algo(hyper_exp)
    res.append(out)
var_mean(res)

ic| mean + first*second: 0.13935841370011234
ic| mean: 0.13936363636363636
ic| mean + first*third: 0.13936885902716037
